# CSE475 Project Bonus, Due: Friday, 05/02/2025

## Instruction

1. Please submit your Jupyter Notebook file (the. ipynb file) containing your code and the outputs produced by your code (note that .ipynb file can contain both the code and the outputs) to Canvas. Please name your file CSE475-ProjectBonus-LastName-FirstName.ipynb.

2. If you have any questions on the homework problems, you should post your question on the Canvas discussion board (under Project Q&A), instead of sending emails to the instructor or TA. We will answer your questions there. In this way, we can avoid repeated questions, and help the entire class stay on the same page whenever any clarification/correction is made.

## Building a Convolutional Neural Network to classify images in the CIFAR-10 Dataset

We will work with the CIFAR-10 Dataset.  This is a well-known dataset for image classification, which consists of 60000 32x32 color images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

The 10 classes are:

<ol start="0">
<li> airplane
<li>  automobile
<li> bird
<li>  cat
<li> deer
<li> dog
<li>  frog
<li>  horse
<li>  ship
<li>  truck
</ol>

For details about CIFAR-10 see:
https://www.cs.toronto.edu/~kriz/cifar.html

For a compilation of published performance results on CIFAR 10, see:
http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html

---

### Building CNN

In this project we will build and train our convolutional neural network. In the first part, we walk through different layers and how they are configured. In the second part, you will build your own model, train it, and compare the performance.

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
## Each image is a 32 x 32 x 3 numpy array
x_train[444].shape

(32, 32, 3)

In [4]:
## Let's look at one of the images

print(y_train[444])
plt.imshow(x_train[444])

[9]


In [5]:
# convert class labels to one-hot vectors
num_classes = 10

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
# see some one-hot vector
y_train[444]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [7]:
# As before, let's make everything float and scale
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## First CNN
Below we will build our first CNN.  For demonstration purpose (so that it will br trained quickly) it is not very deep and has relatively few parameters.  We use strides of 2 in the first two convolutional layers which quickly reduces the dimensions of the output. After a MaxPooling layer, we flatten, and then have a single fully connected layer before the final classification layer.

In [8]:
# Let's build a CNN using Keras' Sequential capabilities

model_1 = Sequential()

## 5x5 convolution with 2x2 stride and 32 filters
model_1.add(Conv2D(32, (5, 5), strides = (2,2), padding='same',
                 input_shape=x_train.shape[1:]))
model_1.add(Activation('relu'))

## Another 5x5 convolution with 2x2 stride and 32 filters
model_1.add(Conv2D(32, (5, 5), strides = (2,2)))
model_1.add(Activation('relu'))

## 2x2 max pooling reduces to 3 x 3 x 32
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(Dropout(0.25))

## Flatten turns 3x3x32 into 288x1
model_1.add(Flatten())
model_1.add(Dense(512))
model_1.add(Activation('relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(num_classes))
model_1.add(Activation('softmax'))

model_1.summary()

/Users/atharva7/miniconda3/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 16, 16, 32)     │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 6, 6, 32)       │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 6, 6, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 3, 3, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 3, 3, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 288)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       147,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 10)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 181,162 (707.66 KB)

 Trainable params: 181,162 (707.66 KB)

 Non-trainable params: 0 (0.00 B)

We still have 181K parameters, even though this is a "small" model.


In [9]:
batch_size = 128

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0005, decay=1e-6)

# Let's train the model using RMSprop
model_1.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model_1.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=15,
              validation_data=(x_test, y_test),
              shuffle=True)

/Users/atharva7/miniconda3/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.2408 - loss: 2.0574 - val_accuracy: 0.4201 - val_loss: 1.6110
Epoch 2/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.4043 - loss: 1.6520 - val_accuracy: 0.4670 - val_loss: 1.4688
Epoch 3/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.4500 - loss: 1.5225 - val_accuracy: 0.5240 - val_loss: 1.3340
Epoch 4/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.4842 - loss: 1.4336 - val_accuracy: 0.5423 - val_loss: 1.2896
Epoch 5/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5068 - loss: 1.3775 - val_accuracy: 0.5549 - val_loss: 1.2454
Epoch 6/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.5341 - loss: 1.3073 - val_accuracy: 0.5776 - val_loss: 1.1975
Epoch 7/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.5417 - loss: 1.2803 - val_accuracy: 0.5907 - val_loss: 1.1678
Epoch 8/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5577 - loss: 1.2440 - val_accu

In [10]:
score = model_1.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.0147857666015625
Test accuracy: 0.6460000276565552


## Your task (25pts)

Our previous model (model_1) had the structure:

Conv -> Conv -> MaxPool -> (Flatten) -> Dense -> Final Classification (with activation functions and dropouts)

Please built a different model (named model_2) by trying different structures and different hyperparameters, such as number of neurons, layers, stride, padding, dropout rate, kernel size, learning rate, number of epochs, etc. You can choose to add data augmentation, batch normalization and/or something new.<br>

For example: <br>
A deeper model: Conv -> Conv -> MaxPool -> Conv -> Conv -> MaxPool -> (Flatten) -> Dense -> Final Classification
<br>

Report the best test accuracy achieved. You will be graded on the highest test accuracy achieved:<br>
Test accuracy < Base model (model_1) : 0 - 5pts (Depending on the changes made in model_2)<br>
Base model (model_1) < Test accuracy < 70%: 5 - 10pts (Depending on the changes made in model_2)<br>
70% < Test accuracy < 75%: 15pts<br>
75% < Test accuracy: 25pts <br>

In [11]:
# Let's build a CNN using Keras' Sequential capabilities
model_2 = Sequential()

In [12]:
#write your code here

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LeakyReLU, Activation
from tensorflow.keras.optimizers import RMSprop

# Step 1: We will use the Sequential API to build a CNN model and we will add the first convolutional layer with LeakyReLU activation function and add the max pooling layer as well.
model_2.add(Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3)))
model_2.add(LeakyReLU(alpha=0.1))
model_2.add(MaxPooling2D(pool_size=(2, 2)))

# Step 2: We now add the second convolutional layer again with LeakyReLU activation function and add the max pooling layer as well.
model_2.add(Conv2D(64, (3, 3), padding='same'))
model_2.add(LeakyReLU(alpha=0.1))
model_2.add(MaxPooling2D(pool_size=(2, 2)))

# Step 3: Now we add the final convolutional layer. 
model_2.add(Conv2D(128, (3, 3), padding='same'))
model_2.add(LeakyReLU(alpha=0.1))
model_2.add(MaxPooling2D(pool_size=(2, 2)))

# Step 4: Now we flatten our 3D output to 1D and add 2 dense layers with LeakyReLU activation function.
model_2.add(Flatten())

# Dense Layer 1
model_2.add(Dense(256))
model_2.add(LeakyReLU(alpha=0.1))

# Dense Layer 2
model_2.add(Dense(128))
model_2.add(LeakyReLU(alpha=0.1))

# Step 5: Finally we add the output layer with softmax activation function.
model_2.add(Dense(10))
model_2.add(Activation('softmax'))

# Now we compile the model using RMSprop optimizer and categorical crossentropy loss function.
opt2 = RMSprop(learning_rate=0.0005, decay=1e-6)

model_2.compile(loss='categorical_crossentropy',
                optimizer=opt2,
                metrics=['accuracy'])


/Users/atharva7/miniconda3/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [13]:
from tensorflow.keras.optimizers import Adam

opt2 = RMSprop(learning_rate=0.0005, decay=1e-6)  # You can experiment with lower values like 0.0003 too
model_2.compile(loss='categorical_crossentropy',
                optimizer=opt2,
                metrics=['accuracy'])

In [14]:
model_2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │         1,290 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 10)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 651,978 (2.49 MB)

 Trainable params: 651,978 (2.49 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Compute internal statistics for normalization (if needed)
datagen.fit(x_train)

# Train the model using augmented data
history_2 = model_2.fit(
    datagen.flow(x_train, y_train, batch_size=128),
    epochs=30,
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // 128,
    shuffle=True
)


Epoch 1/30


/Users/atharva7/miniconda3/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


390/390 ━━━━━━━━━━━━━━━━━━━━ 23s 59ms/step - accuracy: 0.2620 - loss: 2.0041 - val_accuracy: 0.4587 - val_loss: 1.4873
Epoch 2/30
  1/390 ━━━━━━━━━━━━━━━━━━━━ 23s 59ms/step - accuracy: 0.4219 - loss: 1.5701

/Users/atharva7/miniconda3/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


390/390 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4219 - loss: 1.5701 - val_accuracy: 0.4876 - val_loss: 1.4639
Epoch 3/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 24s 61ms/step - accuracy: 0.4628 - loss: 1.4882 - val_accuracy: 0.5746 - val_loss: 1.2014
Epoch 4/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5547 - loss: 1.3527 - val_accuracy: 0.5522 - val_loss: 1.2632
Epoch 5/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 24s 61ms/step - accuracy: 0.5409 - loss: 1.2775 - val_accuracy: 0.5933 - val_loss: 1.1301
Epoch 6/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6094 - loss: 1.0709 - val_accuracy: 0.6014 - val_loss: 1.1115
Epoch 7/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 26s 65ms/step - accuracy: 0.5917 - loss: 1.1517 - val_accuracy: 0.6573 - val_loss: 0.9761
Epoch 8/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6250 - loss: 1.0124 - val_accuracy: 0.6607 - val_loss: 0.9592
Epoch 9/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 26s 67ms/step - accuracy: 0.6258 - loss: 1.0574 - val_accuracy: 0.668

In [16]:
# Test the model on test data
score = model_2.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.659517765045166
Test accuracy: 0.7739999890327454


In [17]:
print('Test accuracy:', score[1] * 100, '%')

Test accuracy: 77.39999890327454 %
